In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
from openpyxl import Workbook,load_workbook

In [3]:
import os

In [4]:
from sklearn.metrics import r2_score,mean_squared_error,precision_score,recall_score,accuracy_score,confusion_matrix

In [5]:
os.chdir('../output_data')

In [6]:
df = pd.read_csv('train_v04.csv',header=0)

In [6]:
ignore_cols = ['Symbol','Series','Date','Prev_Close','Open','Close','Prev_Close_min','Prev_Close_max','Sector']

In [7]:
os.chdir('../pickles')

In [8]:
numerical = pickle.load(open('numerical.pickle','rb'))
categorical = pickle.load(open('categorical.pickle','rb'))
conversion_dict = pickle.load(open('conversion_dict.pickle','rb'))
imputation_cols = pickle.load(open('imputation_cols.pickle','rb'))
model_columns = pickle.load(open('model_columns.pickle','rb'))
lr_model_columns = pickle.load(open('lr_model_columns.pickle','rb'))
order = pickle.load(open('order.pickle','rb'))
pca = pickle.load(open('pca.pickle','rb'))
scale = pickle.load(open('scale.pickle','rb'))

In [9]:
from keras.layers import Input,Dense,Dropout

Using TensorFlow backend.


In [10]:
from keras.models import Model
from keras.optimizers import Adam,RMSprop

In [11]:
len(lr_model_columns)

24

In [13]:
states = Input(shape=(24,))
x = Dense(units = 13, activation='linear')(states)
y = Dense(units = 13,activation='linear')(x)
q_value = Dense(units = 1,activation='linear')(y)
ann_model = Model(inputs=states,outputs=q_value)
ann_model.compile(loss='mse',optimizer=Adam(lr=.001))

In [15]:
model_inputs = np.array(df[lr_model_columns])
model_returns = np.array(df['actual_return'])

In [16]:
ann_model.fit(model_inputs,model_returns,epochs=100,batch_size=50)

Epoch 1/100
85799/85799 [==============================] - 63s 739us/step - loss: 0.0025: 6:02 - - ETA: 4: - ETA: 3:35 - loss: 0.029 - ETA: 3:34 - loss - ETA - ETA: 3:04  - ETA: 2:55 - loss: 0.0 - ETA: 2:54 - ETA: 2:32 - l - - ETA: 1:45 - loss - ETA: 1 - - ETA: 20s -  - - ETA:  - ETA: 9s - loss: 0.00 - ETA: 9s - - - ETA: 0s - loss: 0.002
Epoch 2/100
85799/85799 [==============================] - 35s 411us/step - loss: 3.3521e-040s - loss: 3.350
Epoch 3/100
85799/85799 [==============================] - 44s 513us/step - loss: 3.4158e-0424s - lo - ETA: 24s - loss - ETA: 23s - lo - ETA: 23s - loss:  - ETA - ET - ETA: 6s -
Epoch 4/100
85799/85799 [==============================] - 45s 524us/step - loss: 3.4343e-0415s - loss: 3.4433 - ETA: 15s -  - - ETA:  - ETA: 10 - ETA: 8s - loss: 3.414 - ETA: 8s - loss: 3.4185e - ETA: 8s - loss: 3.4197 - ETA: 8s - loss: 3.42 - ETA: 7s - loss: 3. - ETA: 7s - los - ETA: 6s - l - ETA: 3s - loss: 3.4179 - ETA: 3s - loss:  - ETA: 1s - 
Epoch 5/100
85799/8579

In [12]:
os.chdir('../Models/')

In [18]:
ann_model.save('ann_model_lr.h5')

In [13]:
from keras.models import load_model

In [14]:
model = load_model('ann_model_lr.h5')

In [21]:
pred_returns = [x[0] for x in model.predict(model_inputs)]

In [22]:
r2_score(y_true=model_returns,y_pred=pred_returns)

0.0047761544061450945

In [23]:
df['lift_predicted'] = [max(0,np.sign(x)) for x in pred_returns]

In [24]:
accuracy_score(y_true=df['lift'],y_pred=df['lift_predicted'])

0.5330598258720964

In [25]:
precision_score(y_true=df['lift'],y_pred=df['lift_predicted'])

0.5051967116852613

In [26]:
recall_score(y_true=df['lift'],y_pred=df['lift_predicted'])

0.42573669495509314

In [27]:
df['month'] = df['Date'].apply(lambda x : str(x)[5:7])
df['year'] = df['Date'].apply(lambda x: str(x)[0:4])

In [28]:
df['correct_prediction'] = df[['lift','lift_predicted']].apply(lambda x: 1 if int(x[0]) == int(x[1]) else 0,axis=1)

In [29]:
df['precise_prediction'] = df[['lift','lift_predicted']].apply(lambda x: 1 if int(x[0]) == 1 and int(x[1]) == 1 else 0,axis=1)

In [30]:
df_grp = df.groupby(['month','year','Symbol','Sector']).agg({'lift':['count','sum'],'lift_predicted':['sum'],
                                                          'correct_prediction':['sum'],'precise_prediction':['sum']})

In [31]:
df_grp.reset_index(inplace=True)

In [32]:
df_grp.columns = ['month','year','Symbol','Sector','trade_days','price_lift','pred_price_lift','correct_predcitions','precise_predictions']

In [33]:
os.chdir('../Statistics/')

In [34]:
wb = load_workbook('report.xlsx')
writer = pd.ExcelWriter('report.xlsx',engine='openpyxl')
writer.book = wb
df_grp.to_excel(writer,sheet_name='ann_lr_price_stats',index=False)
writer.save()
writer.close()

In [15]:
os.chdir('../Input_Data')

In [16]:
df = pd.read_csv('oot.csv',header=0)

In [17]:
rename_dict = {col : col.replace('%','pct_') for col in df.columns if '%' in col}

In [18]:
df_val = df

In [19]:
df_val.rename(rename_dict,axis=1,inplace=True)

In [20]:
df_val['lift'] = df_val['actual_return'].apply(lambda x: 1 if x >0 else 0) 

In [21]:
for col in categorical:
    for category in conversion_dict[col]:
        df_val[col+'_dum_'+str(category)] = 0
        df_val.loc[df_val[col]==category,col+'_dum_'+str(category)]=1
    print(col+' done')

Sector done


In [22]:
rename_dict = {col : col.replace(' ','_').replace('-','_').replace('&','_') for col in df.columns if '-' in col or ' ' in col}

In [23]:
df_val.rename(rename_dict,axis=1,inplace=True)

In [24]:
for col in model_columns:
    df_val[col] = df_val[col].apply(lambda x : x if np.abs(x) <= 1 else np.sign(x)*1)

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 24)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 13)                325       
_________________________________________________________________
dense_2 (Dense)              (None, 13)                182       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 14        
Total params: 521
Trainable params: 521
Non-trainable params: 0
_________________________________________________________________


In [26]:
def get_equation(model):
    mat1 = np.matrix(model.layers[1].get_weights()[0])
    mat2 = np.matrix(model.layers[1].get_weights()[1])
    mat3 = np.concatenate((mat1,mat2),axis=0)
    mat4 = np.matrix(model.layers[2].get_weights()[0])
    mat5 = np.matmul(mat3,mat4)
    mat5[13,:]+=np.matrix(model.layers[2].get_weights()[1])
    mat6 = np.matrix(model.layers[3].get_weights()[0])
    mat7 = np.matmul(mat5,mat6)
    mat7[13,:]+=np.matrix(model.layers[3].get_weights()[1])
    
    return [float(x) for x in mat7[:,0]]

In [27]:
dynamic_eqn = pd.DataFrame()
dynamic_eqn['Variable'] = list(lr_model_columns)+['intercept']
dynamic_eqn['eq0'] = get_equation(model)

In [51]:
model.predict(np.array(model_inputs[58,:]).reshape(1,24))[0][0]

-0.0026325528

In [52]:
np.dot(np.array(list(model_inputs[58,:])+[1]),np.array(get_equation(model)))

-0.0022582679733200205

In [28]:
dates = sorted(list(df_val['Date'].unique()))

In [29]:
os.chdir('../Models/')

In [30]:
model = load_model('ann_model_lr.h5')

In [31]:
ann_val_stats = pd.DataFrame()
for i,date in enumerate(dates):
    df_req = df_val[df_val['Date']==date]
    model_inputs = np.array(df_req[lr_model_columns])
    model_returns = np.array(df_req['actual_return'])
    df_req['pred_return'] = [x[0] for x in model.predict(model_inputs)]
    df_req['pred_lift'] = df_req['pred_return'].apply(lambda x : 1 if x >0 else 0)
    df_append = df_req[['Date','Symbol','Sector','actual_return','lift','pred_return']]
    ann_val_stats = ann_val_stats.append(df_append)
    model.fit(model_inputs,model_returns,batch_size=50,epochs=100)
    model.save('ann_model_lr.h5')
    model = load_model('ann_model_lr.h5')
    dynamic_eqn['eq'+str(i+1)] = get_equation(model)
    print('\n')
    print(dynamic_eqn)
    print('\n')
    print('{} days done'.format(i+1))
    print('\n')

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


Epoch 1/100
50/50 [==============================] - 0s 3ms/step - loss: 3.6323e-04
Epoch 2/100
50/50 [==============================] - 0s 80us/step - loss: 3.5548e-04
Epoch 3/100
50/50 [==============================] - 0s 100us/step - loss: 3.4368e-04
Epoch 4/100
50/50 [==============================] - 0s 60us/step - loss: 3.3199e-04
Epoch 5/100
50/50 [==============================] - 0s 419us/step - loss: 3.2357e-04
Epoch 6/100
50/50 [==============================] - 0s 80us/step - loss: 3.1979e-04
Epoch 7/100
50/50 [==============================] - 0s 40us/step - loss: 3.2024e-04
Epoch 8/100
50/50 [==============================] - 0s 40us/step - loss: 3.2325e-04
Epoch 9/100
50/50 [==============================] - 0s 160us/step - loss: 3.2678e-04
Epoch 10/100
50/50 [==============================] - 0s 40us/step - loss: 3.2912e-04
Epoch 11/100
50/50 [==============================] - 0s 60us/step - loss: 3.2936e-04
Epoch 12/100
50/50 [==============================] - 0s 60us

Epoch 97/100
50/50 [==============================] - 0s 60us/step - loss: 3.0558e-04
Epoch 98/100
50/50 [==============================] - 0s 60us/step - loss: 3.0545e-04
Epoch 99/100
50/50 [==============================] - 0s 40us/step - loss: 3.0532e-04
Epoch 100/100
50/50 [==============================] - 0s 100us/step - loss: 3.0518e-04


                         Variable       eq0       eq1
0          Volume_auto_pct_change -0.000143  0.001439
1           Close_fmcg_pct_change -0.073676 -0.012850
2             Close_it_pct_change -0.059378 -0.003632
3            Low_media_pct_change -0.073905 -0.030684
4         Volume_media_pct_change -0.003446 -0.002586
5      Turnover_realty_pct_change -0.000746 -0.002395
6                  return_5d_fmcg -0.026140  0.007979
7                    return_2d_it  0.084764  0.000390
8                    return_3d_it  0.053480 -0.004234
9                    return_4d_it  0.039011  0.002977
10                   return_5d_it -0.113091 -0.025817
11  

Epoch 76/100
50/50 [==============================] - 0s 60us/step - loss: 1.9566e-04
Epoch 77/100
50/50 [==============================] - 0s 60us/step - loss: 1.9507e-04
Epoch 78/100
50/50 [==============================] - 0s 60us/step - loss: 1.9451e-04
Epoch 79/100
50/50 [==============================] - 0s 40us/step - loss: 1.9396e-04
Epoch 80/100
50/50 [==============================] - 0s 80us/step - loss: 1.9343e-04
Epoch 81/100
50/50 [==============================] - 0s 20us/step - loss: 1.9292e-04
Epoch 82/100
50/50 [==============================] - 0s 60us/step - loss: 1.9242e-04
Epoch 83/100
50/50 [==============================] - 0s 140us/step - loss: 1.9195e-04
Epoch 84/100
50/50 [==============================] - 0s 40us/step - loss: 1.9149e-04
Epoch 85/100
50/50 [==============================] - 0s 60us/step - loss: 1.9106e-04
Epoch 86/100
50/50 [==============================] - 0s 60us/step - loss: 1.9064e-04
Epoch 87/100
50/50 [==============================] -

Epoch 52/100
50/50 [==============================] - 0s 40us/step - loss: 1.8757e-04
Epoch 53/100
50/50 [==============================] - 0s 40us/step - loss: 1.8740e-04
Epoch 54/100
50/50 [==============================] - 0s 60us/step - loss: 1.8724e-04
Epoch 55/100
50/50 [==============================] - 0s 40us/step - loss: 1.8710e-04
Epoch 56/100
50/50 [==============================] - 0s 60us/step - loss: 1.8698e-04
Epoch 57/100
50/50 [==============================] - 0s 40us/step - loss: 1.8687e-04
Epoch 58/100
50/50 [==============================] - 0s 60us/step - loss: 1.8677e-04
Epoch 59/100
50/50 [==============================] - 0s 80us/step - loss: 1.8668e-04
Epoch 60/100
50/50 [==============================] - 0s 60us/step - loss: 1.8659e-04
Epoch 61/100
50/50 [==============================] - 0s 40us/step - loss: 1.8650e-04
Epoch 62/100
50/50 [==============================] - 0s 40us/step - loss: 1.8641e-04
Epoch 63/100
50/50 [==============================] - 

...

Epoch 1/100
50/50 [==============================] - 1s 27ms/step - loss: 1.5151e-04
Epoch 2/100
50/50 [==============================] - 0s 219us/step - loss: 1.5077e-04
Epoch 3/100
50/50 [==============================] - 0s 219us/step - loss: 1.4947e-04
Epoch 4/100
50/50 [==============================] - 0s 220us/step - loss: 1.4782e-04
Epoch 5/100
50/50 [==============================] - 0s 239us/step - loss: 1.4598e-04
Epoch 6/100
50/50 [==============================] - 0s 239us/step - loss: 1.4410e-04
Epoch 7/100
50/50 [==============================] - 0s 239us/step - loss: 1.4226e-04
Epoch 8/100
50/50 [==============================] - 0s 239us/step - loss: 1.4049e-04
Epoch 9/100
50/50 [==============================] - 0s 279us/step - loss: 1.3881e-04
Epoch 10/100
50/50 [==============================] - 0s 180us/step - loss: 1.3720e-04
Epoch 11/100
50/50 [==============================] - 0s 199us/step - loss: 1.3564e-04
Epoch 12/100
50/50 [==============================] -

Epoch 96/100
50/50 [==============================] - 0s 160us/step - loss: 1.0668e-04
Epoch 97/100
50/50 [==============================] - 0s 259us/step - loss: 1.0659e-04
Epoch 98/100
50/50 [==============================] - 0s 160us/step - loss: 1.0649e-04
Epoch 99/100
50/50 [==============================] - 0s 160us/step - loss: 1.0640e-04
Epoch 100/100
50/50 [==============================] - 0s 160us/step - loss: 1.0630e-04


                         Variable       eq0       eq1       eq2       eq3  \
0          Volume_auto_pct_change -0.000143  0.001439  0.008040 -0.001102   
1           Close_fmcg_pct_change -0.073676 -0.012850 -0.003057  0.020928   
2             Close_it_pct_change -0.059378 -0.003632  0.005106  0.023972   
3            Low_media_pct_change -0.073905 -0.030684 -0.019433  0.033103   
4         Volume_media_pct_change -0.003446 -0.002586 -0.009997 -0.000826   
5      Turnover_realty_pct_change -0.000746 -0.002395 -0.002971 -0.000743   
6                  retu

Epoch 14/100
50/50 [==============================] - 0s 239us/step - loss: 1.1062e-04
Epoch 15/100
50/50 [==============================] - 0s 239us/step - loss: 1.1001e-04
Epoch 16/100
50/50 [==============================] - 0s 239us/step - loss: 1.0936e-04
Epoch 17/100
50/50 [==============================] - 0s 219us/step - loss: 1.0875e-04
Epoch 18/100
50/50 [==============================] - 0s 219us/step - loss: 1.0820e-04
Epoch 19/100
50/50 [==============================] - 0s 199us/step - loss: 1.0774e-04
Epoch 20/100
50/50 [==============================] - 0s 199us/step - loss: 1.0738e-04
Epoch 21/100
50/50 [==============================] - 0s 259us/step - loss: 1.0710e-04
Epoch 22/100
50/50 [==============================] - 0s 219us/step - loss: 1.0687e-04
Epoch 23/100
50/50 [==============================] - 0s 239us/step - loss: 1.0667e-04
Epoch 24/100
50/50 [==============================] - 0s 199us/step - loss: 1.0646e-04
Epoch 25/100
50/50 [=======================

Epoch 1/100
50/50 [==============================] - 2s 31ms/step - loss: 2.9411e-04
Epoch 2/100
50/50 [==============================] - 0s 199us/step - loss: 2.8212e-04
Epoch 3/100
50/50 [==============================] - 0s 180us/step - loss: 2.6264e-04
Epoch 4/100
50/50 [==============================] - 0s 199us/step - loss: 2.4114e-04
Epoch 5/100
50/50 [==============================] - 0s 199us/step - loss: 2.2249e-04
Epoch 6/100
50/50 [==============================] - 0s 199us/step - loss: 2.0987e-04
Epoch 7/100
50/50 [==============================] - 0s 219us/step - loss: 2.0434e-04
Epoch 8/100
50/50 [==============================] - 0s 239us/step - loss: 2.0504e-04
Epoch 9/100
50/50 [==============================] - 0s 219us/step - loss: 2.0978e-04
Epoch 10/100
50/50 [==============================] - 0s 219us/step - loss: 2.1585e-04
Epoch 11/100
50/50 [==============================] - 0s 199us/step - loss: 2.2075e-04
Epoch 12/100
50/50 [==============================] -

Epoch 96/100
50/50 [==============================] - 0s 239us/step - loss: 1.7845e-04
Epoch 97/100
50/50 [==============================] - 0s 200us/step - loss: 1.7842e-04
Epoch 98/100
50/50 [==============================] - 0s 219us/step - loss: 1.7840e-04
Epoch 99/100
50/50 [==============================] - 0s 279us/step - loss: 1.7837e-04
Epoch 100/100
50/50 [==============================] - 0s 439us/step - loss: 1.7834e-04


                         Variable       eq0       eq1       eq2       eq3  \
0          Volume_auto_pct_change -0.000143  0.001439  0.008040 -0.001102   
1           Close_fmcg_pct_change -0.073676 -0.012850 -0.003057  0.020928   
2             Close_it_pct_change -0.059378 -0.003632  0.005106  0.023972   
3            Low_media_pct_change -0.073905 -0.030684 -0.019433  0.033103   
4         Volume_media_pct_change -0.003446 -0.002586 -0.009997 -0.000826   
5      Turnover_realty_pct_change -0.000746 -0.002395 -0.002971 -0.000743   
6                  retu

In [32]:
len(dates)

183

In [33]:
os.chdir('../Statistics/')

In [59]:
wb = load_workbook('report.xlsx')
writer = pd.ExcelWriter('report.xlsx',engine='openpyxl')
writer.book = wb
dynamic_eqn.to_excel(writer,sheet_name='ann_dyn_eqn_lr',index=False)
writer.save()
writer.close()

In [34]:
ann_val_stats['month'] = ann_val_stats['Date'].apply(lambda x : str(x)[5:7])
ann_val_stats['year'] = ann_val_stats['Date'].apply(lambda x: str(x)[0:4])

In [35]:
ann_val_stats['pred_lift'] = ann_val_stats['pred_return'].apply(lambda x : 1 if x >0 else 0)

In [36]:
ann_val_stats['correct_prediction'] = ann_val_stats[['lift','pred_lift']].apply(lambda x: 1 if int(x[0]) == int(x[1]) else 0,axis=1)

In [37]:
ann_val_stats['precise_prediction'] = ann_val_stats[['lift','pred_lift']].apply(lambda x: 1 if int(x[0]) == 1 and int(x[1]) == 1 else 0,axis=1)

In [38]:
ann_val_grp = ann_val_stats.groupby(['month','year','Symbol','Sector']).agg({'lift':['count','sum'],'pred_lift':['sum'],
                                                          'correct_prediction':['sum'],'precise_prediction':['sum']})

In [39]:
ann_val_grp.reset_index(inplace=True)

In [40]:
ann_val_grp.columns = ['month','year','Symbol','Sector','trade_days','price_lift','pred_price_lift','correct_predcitions','precise_predictions']

In [41]:
ann_val_grp

,month,year,Symbol,Sector,trade_days,price_lift,pred_price_lift,correct_predcitions,precise_predictions
0,01,2020,ADANIPORTS,Services - Shipping,2,2,0,0,0
1,01,2020,ASIANPAINT,Consumer Goods,2,1,0,1,0
2,01,2020,AXISBANK,Banking,2,1,0,1,0
3,01,2020,BAJAJ-AUTO,Automobile,2,0,0,2,0
4,01,2020,BAJAJFINSV,Financial Services,2,1,0,1,0
5,01,2020,BAJFINANCE,Financial Services,2,1,0,1,0
6,01,2020,BHARTIARTL,Telecommunication,2,1,1,2,1
7,01,2020,BPCL,Energy - Oil & Gas,2,0,1,1,0
8,01,2020,BRITANNIA,Consumer Goods,2,1,0,1,0
9,01,2020,CIPLA,Pharmaceuticals,2,0,1,1,0


In [42]:
wb = load_workbook('report.xlsx')
writer = pd.ExcelWriter('report.xlsx',engine='openpyxl')
writer.book = wb
ann_val_grp.to_excel(writer,sheet_name='ann_oot_stats_lr',index=False)
writer.save()
writer.close()

In [43]:
os.chdir('../Models/')

In [44]:
model.save('ann_model_lr.h5')